# Create a combined file for all toys across all departments

In [1]:
import pandas as pd

## List Comprehension

In [2]:
nums = [ 1, 2 , 4, 8, 10 ]
squares = list()
for num in nums:
    squares.append(num ** 2)

squares

[1, 4, 16, 64, 100]

In [3]:
nums = [ 1, 2 , 4, 8, 10 ]
squares = [ num ** 2 for num in nums ]
squares

[1, 4, 16, 64, 100]

In [4]:
[ num + 10  for num in nums ]

[11, 12, 14, 18, 20]

In [5]:
[ num + 10  for num in nums if num > 5 ]

[18, 20]

## Shell Globbing

- List all the pdf files ?
- `dir *.pdf` : * : Globbing character
- `dir sales_*.xls` : All files with sales_ at the start and having xls extension



In [6]:
from glob import glob

In [7]:
glob('Data/*.csv')

['Data/outdoor_rev.csv',
 'Data/board_exp.csv',
 'Data/legos_rev.csv',
 'Data/puzzles_exp.csv',
 'Data/outdoor_exp.csv',
 'Data/puzzles_rev.csv',
 'Data/legos_exp.csv',
 'Data/board_rev.csv',
 'Data/wooden_rev.csv',
 'Data/wooden_exp.csv']

### Get a list of all expenses files ?

In [8]:
glob('Data/*_exp.csv')

['Data/board_exp.csv',
 'Data/puzzles_exp.csv',
 'Data/outdoor_exp.csv',
 'Data/legos_exp.csv',
 'Data/wooden_exp.csv']

## Load all expenses files into individual DataFrames

- For every file in *_exp.csv
- load it using read_csv

In [14]:
df_exps = [ pd.read_csv(file, parse_dates=['Month']) 
             for file in glob('Data/*_exp.csv') 
          ]
len(df_exps)

5

In [17]:
df_exps[0].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Month        12 non-null     datetime64[ns]
 1   Product      12 non-null     object        
 2   Expenditure  12 non-null     int64         
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 420.0+ bytes


In [19]:
df_exps[1]

,Month,Product,Expenditure
0,2014-07-01,Puzzles,18411
1,2014-08-01,Puzzles,15432
2,2014-09-01,Puzzles,303623
3,2014-10-01,Puzzles,19697
4,2014-11-01,Puzzles,269769
5,2014-12-01,Puzzles,49918


In [20]:
df_exps[2]

,Month,Product,Expenditure
0,2014-01-01,Outdoor,154945
1,2014-02-01,Outdoor,20656
2,2014-03-01,Outdoor,102635
3,2014-04-01,Outdoor,68123
4,2014-05-01,Outdoor,22463
5,2014-06-01,Outdoor,165650
6,2014-07-01,Outdoor,256723
7,2014-08-01,Outdoor,78171
8,2014-09-01,Outdoor,12942
9,2014-10-01,Outdoor,99254


In [22]:
dfx = pd.concat( [ pd.read_csv(file, parse_dates=['Month']) 
                        for file in glob('Data/*_exp.csv') 
                    ]
                   )
dfx.head()

,Month,Product,Expenditure
0,2014-01-01,Board,61617
1,2014-02-01,Board,204763
2,2014-03-01,Board,8466
3,2014-04-01,Board,146850
4,2014-05-01,Board,188737


### Combine all the Revenue data into a single DataFrame

In [23]:
dfr = pd.concat( [ pd.read_csv(file, parse_dates=['Month']) 
                        for file in glob('Data/*_rev.csv') 
                    ]
                   )
dfr.head()

,Month,Product,Revenue
0,2014-01-01,Outdoor,18181
1,2014-02-01,Outdoor,153340
2,2014-03-01,Outdoor,110857
3,2014-04-01,Outdoor,23880
4,2014-05-01,Outdoor,65460


In [24]:
dfx.shape

(51, 3)

In [25]:
dfr.shape

(51, 3)

In [27]:
dfx.columns

Index(['Month', 'Product', 'Expenditure'], dtype='object')

In [28]:
dfr.columns

Index(['Month', 'Product', 'Revenue'], dtype='object')

### Sort the values by Product Name and display first 3 rows

In [34]:
dfx.sort_values(['Product', 'Month']).head(3)

,Month,Product,Expenditure
0,2014-01-01,Board,61617
1,2014-02-01,Board,204763
2,2014-03-01,Board,8466


In [35]:
dfr.sort_values(['Product', 'Month']).head(3)

,Month,Product,Revenue
0,2014-01-01,Board,82531
1,2014-02-01,Board,121172
2,2014-03-01,Board,135065


In [46]:
df_toys = pd.merge(dfx, dfr)
df_toys.head()

,Month,Product,Expenditure,Revenue
0,2014-01-01,Board,61617,82531
1,2014-02-01,Board,204763,121172
2,2014-03-01,Board,8466,135065
3,2014-04-01,Board,146850,270177
4,2014-05-01,Board,188737,71614


In [47]:
df_toys.apply(lambda row:f'Q{row.Month.quarter}', axis=1).head()

0    Q1
1    Q1
2    Q1
3    Q2
4    Q2
dtype: object

In [48]:
df_toys.loc[:, 'Quarter'] =  ( df_toys
                              .apply(lambda row:f'Q{row.Month.quarter}', 
                                     axis=1)
                             )

df_toys.head()

,Month,Product,Expenditure,Revenue,Quarter
0,2014-01-01,Board,61617,82531,Q1
1,2014-02-01,Board,204763,121172,Q1
2,2014-03-01,Board,8466,135065,Q1
3,2014-04-01,Board,146850,270177,Q2
4,2014-05-01,Board,188737,71614,Q2


## Create a csv file with Quarter, Product, Exp and Rev columns

- Aggregate Expenditure and Revenue for every Quarter

In [53]:
df_toys = df_toys.drop(columns=['Month'])
df_toys.head()

,Product,Expenditure,Revenue,Quarter
0,Board,61617,82531,Q1
1,Board,204763,121172,Q1
2,Board,8466,135065,Q1
3,Board,146850,270177,Q2
4,Board,188737,71614,Q2


In [52]:
cols = "Quarter Product Expenditure Revenue".split()
cols

['Quarter', 'Product', 'Expenditure', 'Revenue']

In [55]:
df_toys[cols].head()

,Quarter,Product,Expenditure,Revenue
0,Q1,Board,61617,82531
1,Q1,Board,204763,121172
2,Q1,Board,8466,135065
3,Q2,Board,146850,270177
4,Q2,Board,188737,71614


In [65]:
( df_toys[cols]
 .groupby(['Quarter', 'Product'])
 .sum()
 .reset_index()
 .sort_values(['Quarter', 'Product'])
 .to_csv("toys.csv", index=False)
)